In [24]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python 

In [105]:
%%time

# -*- coding: utf-8 -*-
"""
Created on Wed Jun 10 15:23:09 2020

@author: jgonzalezh

"""

import pyodbc
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
import time
from pulp import *
import matplotlib.colors as mcolors
import matplotlib.transforms as mtransforms
import matplotlib.patches as mpatches
import math
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.figure_factory import create_gantt




class DATA:
    def __init__(self,archivo_dmd,archivo_equipos,instr):
        self.instrumental=instr
        x=instr.drop(instr.columns[[0, 1]], axis=1).iloc[1]
        y=instr.drop(instr.columns[[0, 1]], axis=1).iloc[0]
        self.Sub_Esp=x.unique()
        self.Esp=y.unique()
        Esp=self.Esp
        shape=instr.shape
        REST3=instr
        for i in range(2,shape[1]):
            for j in range(5,shape[0]):
                REST3.iat[j,i]=instr.fillna(0).iat[j,i]*instr.fillna(0).iat[2,i]/instr.fillna(0).iat[j,1]
        similitudes=[[0 for i in range(len(Esp))]for j in range(len(Esp))]
        columnas_esp=[[] for i in range(len(Esp))]
        for i in range(REST3.shape[1]):
            for j in range(len(Esp)):
                if REST3.iat[0,i]==Esp[j]:
                    columnas_esp[j].append(i)
        #print(columnas_esp)

        instrumentos=[[0 for x in range(5,REST3.shape[0])]for i in range(len(Esp))]
        #print(len(instrumentos))
        #print(len(instrumentos[0]))
        for i in range(5,REST3.shape[0]):
            for j in range(len(Esp)):
                for z in columnas_esp[j]:
                    instrumentos[j][i-5]+=REST3.iat[i,z]
        #print(instrumentos)
        for i in range(len(Esp)):
                for j in range(len(Esp)):
                    for z in range(len(instrumentos[0])):
                        if instrumentos[j][z]>0 and instrumentos[i][z]>0:
                            similitudes[i][j]+= round(instrumentos[i][z]+instrumentos[j][z],2)
                            similitudes[j][i]+= round(instrumentos[i][z]+instrumentos[j][z],2)

        #print(Esp,'\n',similitudes)
        self.instrumentos=instrumentos
        self.similitudes=similitudes
        self.Especialidades=archivo_dmd.Especialidad.unique()
        self.Cobertura=archivo_dmd.Tipo.unique() 
        a=len(self.Especialidades)
        b=len(self.Cobertura)
        self.N_Especialidad=a
        self.N_Cobertura=b
        self.Dmd_semanal=np.zeros(shape=(a,b))
        self.Bloquesmax= np.zeros(shape=(a,b))
        for i in range(len(archivo_dmd.index)):
            for e in range(a):
                for c in range(b):
                    if archivo_dmd.iat[i,0]==self.Especialidades[e]:
                        if archivo_dmd.iat[i,1]==self.Cobertura[c]:
                            self.Dmd_semanal[e,c]=archivo_dmd.iat[i,4]
                            self.Bloquesmax[e,c]=archivo_dmd.iat[i,3]
                            
                            
        #Turnos equipos médicos 
        dias=['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo'] 
        #Tipo=['Espontaneo', 'Controles']
        horas=['00:00:00','00:30:00','01:00:00','01:30:00','02:00:00','02:30:00','03:00:00','03:30:00',
            '04:00:00','04:30:00','05:00:00','05:30:00','06:00:00','06:30:00','07:00:00','07:30:00',
            '08:00:00','08:30:00','09:00:00','09:30:00','10:00:00','10:30:00','11:00:00','11:30:00',
            '12:00:00','12:30:00','13:00:00','13:30:00','14:00:00','14:30:00','15:00:00','15:30:00',
            '16:00:00','16:30:00','17:00:00','17:30:00','18:00:00','18:30:00','19:00:00','19:30:00',
            '20:00:00','20:30:00','21:00:00','21:30:00','22:00:00','22:30:00','23:00:00','23:30:00']
        FTE=1
        self.turno=archivo_equipos 
        turnos=self.turno
        fill=len(turnos.index)
        
        turnos['Hora Inicio']=turnos['Hora inicio Turno'].dt.hour
        turnos['Minutos Inicio']=turnos['Hora inicio Turno'].dt.minute
        turnos['Hora Fin']=turnos['Hora Fin Turno'].dt.hour
        turnos['Minutos Fin']=turnos['Hora Fin Turno'].dt.minute
        #turnos['Hora Colacion In']=turnos['Hora Inicio Colación'].dt.hour
        #turnos['Minutos colacion In']=turnos['Hora Inicio Colación'].dt.minute
        #turnos['Hora Colacion fin']=turnos['Hora Fin Colación'].dt.hour
        #turnos['Minutos colacion fin']=turnos['Hora Fin Colación'].dt.minute
        self.T=[[[0 for i in range(48)]for j in range(7)]for e in range(self.N_Especialidad)]
        
        for g in range(fill):
            for e in  range(self.N_Especialidad):
                if turnos.iat[g,0]==self.Especialidades[e]:
                    for h in range(7):
                        if turnos.iat[g,2]==dias[h]:
                            for j in range(int(round(2*turnos.iat[g,9]+turnos.iat[g,10]/30))):
                                if j>=int(round(2*turnos.iat[g,7]+turnos.iat[g,8]/30)) and j<=int(round(2*turnos.iat[g,9]+(turnos.iat[g,10]/30))): #revisa si tiene que ponerle -1
                                    self.T[e][h][j]+=FTE

def diccionario_bloques(bloque_ini,bloque_fin):
    diccionario=[]
    h=bloque_ini
    g=bloque_fin
    for k in range(h,g):
        for j in range (k,g):    
            largo=[]
            #for i in [16]:#]range(16,42):
            for u in range(k):
                largo.append(0)
            for i in range(k,g):
                if i<=j:
                    largo.append(1)
                else:
                    largo.append(0)
            for p in range(g,48):
                largo.append(0)
            diccionario.append(largo)
    return diccionario
  

Wall time: 1.04 ms


In [106]:
      
DMD=pd.read_excel('C:/Users/jgonzalezh/Documents/Pabellones 2/input/Input_2.xlsx', sheet_name='Horas semanales')
EQ=pd.read_excel('C:/Users/jgonzalezh/Documents/Pabellones 2/input/Horarios médicos_2.xlsx')
REST=pd.read_excel('C:/Users/jgonzalezh/Documents/Pabellones 2/input/Instrumental.xlsx',  header=None )
#EQ.info()
EQ['Hora inicio Turno']=pd.to_datetime(EQ['Hora inicio Turno'], format='%H:%M:%S')
EQ['Hora Fin Turno']=pd.to_datetime(EQ['Hora Fin Turno'], format='%H:%M:%S')

#sheetname : string, int, mixed list of strings/ints, or None, def
data=DATA(DMD,EQ,REST) 

#data.Dmd_semanal[4,1]
                      
#should make dictionary for variables
E_str=data.Especialidades
C_str=data.Cobertura
E=range(data.N_Especialidad)
C=range(data.N_Cobertura)
P=[0,1,2,3,4,5,6,7,8,9,10]#,11]
P_str=['Pabellón 1','Pabellón 2','Pabellón 3','Pabellón 4','Pabellón 5','Pabellón 6','Pabellón 7','Pabellón 8','UCA 1','UCA 2','UCA 3']
horas=['00:00:00','00:30:00','01:00:00','01:30:00','02:00:00','02:30:00','03:00:00','03:30:00',
    '04:00:00','04:30:00','05:00:00','05:30:00','06:00:00','06:30:00','07:00:00','07:30:00',
    '08:00:00','08:30:00','09:00:00','09:30:00','10:00:00','10:30:00','11:00:00','11:30:00',
    '12:00:00','12:30:00','13:00:00','13:30:00','14:00:00','14:30:00','15:00:00','15:30:00',
    '16:00:00','16:30:00','17:00:00','17:30:00','18:00:00','18:30:00','19:00:00','19:30:00',
    '20:00:00','20:30:00','21:00:00','21:30:00','22:00:00','22:30:00','23:00:00','23:30:00']
dias=[0,1,2,3,4] # se hace el ppl de lunes a viernes
index = np.arange(48) #bloques del día
dias_str=['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo'] 

Tipo=[0,1,2,3,4,5,6]
Q_horas=data.Dmd_semanal #ordenadas por especialidad mínimo (vemos a que apuntamos acá)
#Q_portramo=[[14,14,7,21,0,21,7],[15,25,25,10,5,0,5,0],[25,15,25,10,5,0,0,10]]
#Horario de funcionamiento 8:00-21:->16,42
turnos=diccionario_bloques(16,42)
T=range(len(turnos))
#B=[3,4,6]
Cirujanos=data.T
min_sem=data.Bloquesmax
#print(*Cirujanos)
print(data.similitudes)



[[10.419999999999998, 7.36, 3.9000000000000004, 2.3200000000000003, 0.22, 1.1800000000000002, 0.36, 0.36, 0.3], [7.36, 11.979999999999997, 4.46, 0.82, 1.12, 2.3200000000000003, 1.1400000000000001, 1.34, 1.72], [3.9000000000000004, 4.46, 6.439999999999999, 0.82, 0.9000000000000001, 1.92, 0.7, 0.94, 1.2999999999999998], [2.3200000000000003, 0.82, 0.82, 3.6800000000000006, 0, 0.16, 0, 0, 0], [0.22, 1.12, 0.9000000000000001, 0, 3.839999999999999, 1.6000000000000003, 1.0, 1.4, 1.7000000000000002], [1.1800000000000002, 2.3200000000000003, 1.92, 0.16, 1.6000000000000003, 2.68, 1.32, 1.7199999999999998, 2.5000000000000004], [0.36, 1.1400000000000001, 0.7, 0, 1.0, 1.32, 3.6000000000000014, 2.4, 1.34], [0.36, 1.34, 0.94, 0, 1.4, 1.7199999999999998, 2.4, 4.780000000000001, 1.92], [0.3, 1.72, 1.2999999999999998, 0, 1.7000000000000002, 2.5000000000000004, 1.34, 1.92, 2.68]]


In [107]:
#print(data.Especialidades[8],data.T[8][4],'\n',data.Especialidades[4],data.T[4][4])

In [108]:
print(data.Esp)
print(E_str)

def posicion(a,b):
    leng=len(b)
    x=None
    for i in range(leng):
        if a==b[i]:
            x=i
    return x
def score(e,e2, b):
    x=posicion(e,b)
    y=posicion(e2,b)
    if x!=None and y!=None:
        z=data.similitudes[x][y]
    else:
        z=0
    return z
"""
for d in dias:
    for e in E:
        for e2 in E:
            print(score(E_str[e],E_str[e2],data.Esp)    )
#posicion(E_str[0],data.Esp)
"""
Q_horas

['Neuro' 'Columna' 'Maxilo' 'CPQ' 'Hombro' 'EESS' 'Cadera' 'Rodilla' 'TyP']
['EESS' 'Rodilla' 'CPQ' 'TyP' 'Columna' 'Maxilo' 'Hombro' 'Libre' 'Neuro'
 'Cadera' 'Roa']


array([[69.25      , 10.        ],
       [50.58333333, 29.41666667],
       [42.16666667, 10.        ],
       [46.58333333, 16.625     ],
       [14.70833333, 15.        ],
       [20.        ,  0.        ],
       [21.83333333, 13.79166667],
       [27.        , 40.        ],
       [ 0.        ,  3.5       ],
       [11.95833333, 15.        ],
       [ 0.        , 12.        ]])

In [ ]:
%%time

#Definir problema
modelo=LpProblem('Bloques de pabellones', LpMinimize) # Se define el modelo
#Variables
A=LpVariable.dicts('Bloques propuestos', [(e,p,t,d,c)for t in T for d in dias for p in P for e in E for c in C],lowBound=0, upBound=None,cat='Binary')
#Topes=LpVariable.dicts('Bloques propuestos', [(e,e2,d) for d in dias for e in E for e2 in E],lowBound=0, upBound=None,cat='Binary')
#Función objetivo 
x=0

x+=pulp.lpSum([A[(e,p,t,d,c)] for d in dias for t in T for p in P for e in E for c in C]) #minimizar N° Bloques a la semana, Se quiere utilizar al máximo pabellones (maximizar la asignación?)


#for t in T 

#Se suman los pabellones que están asignados de una especialidad con respecto a la otra el msimo día y se multiplica por su similitud
#la idea es que si se asignan muchos pabellones de instrumentales similares castigue en la función objetivo
#Para cuando queda en la misma especialidad también castiga, así no se sobrecarga un mismo día inecesariamente
#ideal seria multiplicar, para si son cero no se castigue a las especialidades con alto puntaje
#quizás se podría ocupar la funcion sigmoidal para normalizar los pesos
"""

"""

for d in dias:
    for e in E:
        for e2 in E:
            x+=(pulp.lpSum([A[(e,p,t,d,c)] + A[(e2,p,t,d,c)] for p in P for c in C for t in T]))*score(E_str[e],E_str[e2],data.Esp)*(1/30)
"""
"""
modelo+=x
#Restricciones

"""
for d in dias:
    for e in E:
        for e2 in E:
            modelo+=pulp.lpSum([A[(e,p,t,d,c)] for p in P for c in C])*pulp.lpSum([A[(e2,p,t,d,c)] for p in P for c in C])*0.00001<Topes(e,e2,d)
"""
# Solo se asigna a una especialidad al mismo tiempo el pabellón
for  p in P:
    for d in dias:
        for h in index:
            modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for e in E for t in T for c in C])<=1 
# Se satisface la demanda semanal
for e in E:
    for c in C:
        modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for h in index for d in dias for p in P])>=Q_horas[e,c]*2
# Bloque es continuo abordada por diccionario
# No hay más de tres bloques por día de cada especialidad, ley o VSC
# Acá se puede hacer un vector que contega 1 en la mañana, otro en la tarde del largo de index y se puede castigar en la Fn obj si VSC es en la mañana y si Ley es en la tarde
for d in dias:
    for e in E:
        modelo+=pulp.lpSum([A[(e,p,t,d,c)] for t in T for p in P for c in C])<=3   
#Bloques mínimos semanales, si esq tienen demanda, ROA ley no tiene por ejemplo
for e in E:
    for c in C:
        if Q_horas[e,c]>0:
            modelo+=pulp.lpSum([A[(e,p,t,d,c)] for t in T for d in dias for p in P])>=min_sem[e,c]#B[e] 
# No hay más de tres bloques en simultaneo (abordada por previa, si son numeros distintos se debe hacer)
"""
for d in dias:
    for e in E:
        for h in index:
            modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d)] for t in T for p in P])<=3   
"""
# No hay más bloques que la capacidad de equipos en los bloques ley

for d in dias:
    for e in E:
        for h in index:
            for c in [0]:
                if E_str[e]!='Libre':
                    modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for p in P])<=Cirujanos[e][d][h] #hay que dividr por el promedio de la especialidad o alguna regla   

# No se asignan más pabellones de los disponibles (ya restringidos por N de pabellones).
# A futuro poner largo máximo de un bloque(?) Puede ser en el diccionario de turnos tb para simplificar problema.
#Largo máximo
for  p in P:
    for d in dias:
        for e in E:
            for c in C:
                for t in T:
                    modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for h in index])<=18
#largo mínimo
"""
"""
for  p in P:
    for d in dias:
        for e in E:
            for c in C:
                for t in T:
                    modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for h in index])>=pulp.lpSum([5*A[(e,p,t,d,c)] for t in T for h in index])
            """
                           if Q_horas[e,c]>0:
           """
# Dividiendo el instrumental se puede hacer todas las cirugías a la semana (cajas) --->FUTURA, hay que modelar los datos

modelo.solve(pulp.GLPK())#pulp.PULP_CBC_CMD(maxSeconds=25000, msg=0, fracGap=0))#


In [ ]:
print(LpStatus[modelo.status])
print(value(modelo.objective))
#print(turnos[1][1])

In [96]:

Result= [[[[[0 for d in dias] for t in T]for p in P ]for c in C]for e in E]
for e in E:
    for p in P:
        for t in T:
            for d in dias:
                for c in C:
                    Result[e][c][p][t][d]=A[(e,p,t,d,c)].varValue
                    if A[(e,p,t,d,c)].varValue==1:
                        print('E', E_str[e],'C',C_str[c], P_str[p],'T',t,'\nTurno',turnos[t][:],'\nDia',d)



E EESS C VSC UCA 2 T 161 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
Dia 2
E Maxilo C Ley Pabellón 1 T 252 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
Dia 0
E Maxilo C Ley Pabellón 3 T 88 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
Dia 3
E Libre C VSC Pabellón 5 T 315 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
Dia 3
E Cadera C Ley Pabellón 8 T 186 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
Dia 0
E Roa C VSC Pabellón 5 T 323 
Turno [0, 0, 0, 0, 0, 0,

In [97]:
import math

#for h in index[1:-1]:
#    modelo
 #!usr/bin/env python
  #coding: utf-8
list=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
def search_ini(list):
    for i in range(48):
        if list[i-1]==0 and list[i]==1:
            z=i
    return z
def search_fin(list):
    for i in range(48):
        if list[i-1]==1 and list[i]==0:
            z=i
    return z
e=search_ini(list)
f=search_fin(list)

def turno(lista):
    #lista=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
    largo=len(lista)
    i=0
    while lista[i]==0:  
        i+=1
    inicio=i
    while lista[i]==1 or lista[i+1]==1:
        fin=i
        i+=1
    if math.modf((inicio)/2)[0] ==0:
        inicio_str=str(int(math.modf((inicio)/2)[1]))+':00:00'
    else:
        inicio_str=str(int(math.modf((inicio)/2)[1]))+':30:00'
    if math.modf((fin)/2)[0] ==0:
        fin_str=str(int(math.modf((fin)/2)[1]))+':30:00'
    else:
        fin_str=str(int(math.modf(((fin)/2)+1)[1]))+':00:00'
    return [inicio_str,fin_str]
                  

In [98]:
def colores(largo):
    colores=['rgb(220, 0, 0)','rgb(220, 55, 0)','rgb(220, 110, 0)','rgb(220, 175, 0)','rgb(220, 220, 0)','rgb(55, 0, 0)',
                'rgb(110, 0, 0)','rgb(220, 0, 0)','rgb(175, 0, 0)','rgb(220, 0, 110)','rgb(220, 55, 110)','rgb(220, 110, 110)',
                'rgb(220, 175, 110)','rgb(220, 220, 110)','rgb(55, 0, 110)','rgb(110, 0, 110)','rgb(220, 0, 110)',
                'rgb(175, 0, 110)','rgb(220, 0, 220)','rgb(220, 55, 220)','rgb(220, 110, 220)',
                'rgb(220, 175, 220)','rgb(220, 220, 220)','rgb(55, 0, 220)','rgb(110, 0, 220)','rgb(220, 0, 220)',
                'rgb(175, 0, 220)']
    return colores[:largo]
df0=[]
df1=[]
df2=[]
df3=[]
df4=[]

resource_0=[]
resource_1=[]
resource_2=[]
resource_3=[]
resource_4=[]
for e in E:
    for p in P:
        for t in T:
            for d in dias:
                for c in C:
                    if Result[e][c][p][t][d]==1:
                        if d==0:

                            resource_0.append(E_str[e]+' '+C_str[c])
                            #df.append(dict(Task=P_str[p], Start=turno(turnos[t])[0],Finish=turno(turnos[t])[1]))
                            df0.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       
                        elif d==1:

                            resource_1.append(E_str[e]+' '+C_str[c])
                            #df.append(dict(Task=P_str[p], Start=turno(turnos[t])[0],Finish=turno(turnos[t])[1]))
                            df1.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       
                        elif d==2:

                            resource_2.append(E_str[e]+' '+C_str[c])
                            #df.append(dict(Task=P_str[p], Start=turno(turnos[t])[0],Finish=turno(turnos[t])[1]))
                            df2.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       
                        elif d==3:

                            resource_3.append(E_str[e]+' '+C_str[c])
                            #df.append(dict(Task=P_str[p], Start=turno(turnos[t])[0],Finish=turno(turnos[t])[1]))
                            df3.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       
                        elif d==4:

                            resource_4.append(E_str[e]+' '+C_str[c])
                            #df.append(dict(Task=P_str[p], Start=turno(turnos[t])[0],Finish=turno(turnos[t])[1]))
                            df4.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       

#print(len(resource))  
A_0=colores(len(resource_0))
colors_0={}

for i in range(len(resource_0)):
    colors_0[resource_0[i]]=A_0[i]
    
A_1=colores(len(resource_1))
colors_1={}

for i in range(len(resource_1)):
    colors_1[resource_1[i]]=A_1[i]
    
A_2=colores(len(resource_2))
colors_2={}

for i in range(len(resource_2)):
    colors_2[resource_2[i]]=A_2[i]
    
    
A_3=colores(len(resource_3))
colors_3={}

for i in range(len(resource_3)):
    colors_3[resource_3[i]]=A_3[i]
    
    
A_4=colores(len(resource_4))
colors_4={}

for i in range(len(resource_4)):
    colors_4[resource_4[i]]=A_4[i]

In [99]:
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.figure_factory import create_gantt


    
"""colors = {resource[0]: 'rgb(220, 0, 0)',
          resource[1]: (1, 0.9, 0.16),
          resource[2]: 'rgb(0, 255, 100)',
          resource[3]: 'rgb(255, 0, 0)',
          resource[4]: 'rgb(170, 14, 200)',
          resource[5]: (1, 1, 0.2),
          resource[6]: (0, 0.5, 0.7),
          resource[7]: (0, 1, 1),
          resource[8]: (0, 1, 0.5),
          resource[9]: (0.5, 1, 0.7),
          resource[10]: (1, 1, 0.7),
          resource[11]: (0, 0, 0.7),
          resource[12]: (0.1, 0, 0.1),
          resource[13]: (0, 0.3, 0.7),
          resource[14]: (0.5, 0, 0.3),
          resource[15]: (0.1, 0.5, 0.7),
          resource[16]: (0.9, 0.3, 0.7),}
          #resource[17]: (0.9, 1, 0.7)}
"""

fig_0 = ff.create_gantt(df0, colors=colors_0, index_col='Resource', show_colorbar=True, title='Día Lunes',showgrid_x=True, showgrid_y=True,
                      group_tasks=True)
fig_0.show()







In [100]:
fig_1 = ff.create_gantt(df1, colors=colors_1, index_col='Resource', show_colorbar=True, title='Día Martes',showgrid_x=True, showgrid_y=True,
                      group_tasks=True)
fig_1.show()



PlotlyError: Your list is empty. It must contain at least one dictionary.

In [101]:
fig_2 = ff.create_gantt(df2, colors=colors_2, index_col='Resource', show_colorbar=True, title='Día Miércoles',showgrid_x=True, showgrid_y=True,
                      group_tasks=True)
fig_2.show()




In [102]:
fig_3 = ff.create_gantt(df3, colors=colors_3, index_col='Resource', show_colorbar=True, title='Día Jueves',showgrid_x=True, showgrid_y=True,
                      group_tasks=True)
fig_3.show()



In [103]:
fig_4 = ff.create_gantt(df4, colors=colors_4, index_col='Resource', show_colorbar=True, title='Día Viernes',showgrid_x=True, showgrid_y=True,
                      group_tasks=True)
fig_4.show()

PlotlyError: Your list is empty. It must contain at least one dictionary.

In [92]:
data.Especialidades

array(['EESS', 'Rodilla', 'CPQ', 'TyP', 'Columna', 'Maxilo', 'Hombro',
       'Libre', 'Neuro', 'Cadera', 'Roa'], dtype=object)